In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sentence_transformers import SentenceTransformer


In [2]:
df = pd.read_csv("tg.csv")

In [3]:
df.head()

,message,source,date,views
0,Утвердили график переноса выходных в 2023 году...,https://t.me/consultant_plus,1.661864e+09,9914
1,Ключевую ставку продолжают снижать\n\nС 25 июл...,https://t.me/consultant_plus,1.658490e+09,10070
2,Попробуйте свои силы в рубрике «Вопрос на засы...,https://t.me/consultant_plus,1.657291e+09,7678
3,Попробуйте свои силы в рубрике «Вопрос на засы...,https://t.me/consultant_plus,1.656681e+09,8092
4,Попробуйте свои силы в рубрике «Вопрос на засы...,https://t.me/consultant_plus,1.654262e+09,8388


## init embedder

In [4]:
embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')


In [7]:
df["message"]

0       Утвердили график переноса выходных в 2023 году...
1       Ключевую ставку продолжают снижать\n\nС 25 июл...
2       Попробуйте свои силы в рубрике «Вопрос на засы...
3       Попробуйте свои силы в рубрике «Вопрос на засы...
4       Попробуйте свои силы в рубрике «Вопрос на засы...
                              ...                        
2834    Напоминаем, что в этом году организации и физи...
2835    #Важно_знать , что теперь помимо полной выписк...
2836    С 1 января 2022 года доходы в виде компенсации...
2837    ФНС России подвела итоги работы по предоставле...
2838    Предлагаем узнать больше о главных изменениях ...
Name: message, Length: 2839, dtype: object

In [8]:
corpus_embeddings = embedder.encode(df["message"])
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)


In [9]:
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

In [13]:
df.message[0]

'Утвердили график переноса выходных в 2023 году\n\nПраздничные дни 1 и 8 января совпали с выходными. Их перенесли на 24 февраля и 8 мая. С учетом этого отдохнем:\n31 декабря – 8 января;\n23–26 февраля;\n8 марта;\n29 апреля – 1 мая;\n6-9 мая;\n10-12 июня;\n4-6 ноября.\n\nДокументы:\nИнформация с сайта Правительства РФ от 30.08.2022 \nПостановление Правительства РФ от 29.08.2022 № 1505\n\nПосмотреть информацию о количестве рабочих и выходных дней в 2023 году всегда можно в системе КонсультантПлюс в Производственном календаре для пятидневной и шестидневной рабочей недели.\nВ календаре также приводятся нормы рабочего времени в часах для разных режимов работы.\n\nПолучить бесплатный пробный доступ к КонсультантПлюс https://login.consultant.ru/demo-access/'

In [16]:
clustered_sentences = {}
from pprint import pprint
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []
    clustered_sentences[cluster_id].append(df.message[sentence_id])

##for i, cluster in clustered_sentences.items():
#    pprint(f"Cluster {i+1}")
#    pprint(cluster)
#    pprint("")

'Cluster 4'
['Утвердили график переноса выходных в 2023 году\n'
 '\n'
 'Праздничные дни 1 и 8 января совпали с выходными. Их перенесли на 24 февраля '
 'и 8 мая. С учетом этого отдохнем:\n'
 '31 декабря – 8 января;\n'
 '23–26 февраля;\n'
 '8 марта;\n'
 '29 апреля – 1 мая;\n'
 '6-9 мая;\n'
 '10-12 июня;\n'
 '4-6 ноября.\n'
 '\n'
 'Документы:\n'
 'Информация с сайта Правительства РФ от 30.08.2022 \n'
 'Постановление Правительства РФ от 29.08.2022 № 1505\n'
 '\n'
 'Посмотреть информацию о количестве рабочих и выходных дней в 2023 году '
 'всегда можно в системе КонсультантПлюс в Производственном календаре для '
 'пятидневной и шестидневной рабочей недели.\n'
 'В календаре также приводятся нормы рабочего времени в часах для разных '
 'режимов работы.\n'
 '\n'
 'Получить бесплатный пробный доступ к КонсультантПлюс '
 'https://login.consultant.ru/demo-access/',
 'Минэкономразвития РФ: в 2022 году в Ставропольском крае появятся девять '
 'новых социальных объектов\n'
 'Новости',
 'О текущей ц

In [18]:
len(clustered_sentences.items())

23